In [ ]:
#Importing Libraries

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import matplotlib
matplotlib.rcParams['figure.figsize'] = (12,6)

In [ ]:
#Read the data

data = pd.read_csv('../input/avocado-prices/avocado.csv', index_col = 0)
data.shape

In [ ]:
#Looking top 5 rows

data.head()

In [ ]:
#Checking for null values

data.isnull().sum().sum()

In [ ]:
#Checking for duplicated values

data[data.duplicated()]

In [ ]:
#Checking the unique values for 'type' column

data.type.unique()

In [ ]:
#Checking the unique values for 'region' column

data.region.unique()

In [ ]:
#Checking the distribution of price

data['AveragePrice'].hist(bins = 25)

In [ ]:
data['Date'] = pd.to_datetime(data['Date'])

In [ ]:
#Creating 3 new columns 'Year', 'Month' and 'Day'

data['month'] = data['Date'].apply(lambda x: x.month)
data['day'] = data['Date'].apply(lambda x: x.day)

In [ ]:
data.head()

In [ ]:
#We will not need anymore 'date' column so we can drop it

data.drop('Date', axis = 1, inplace = True)

In [ ]:
data.head()

In [ ]:
year_feature = ['year', 'month', 'day']

for feature in year_feature:
    data.groupby(feature)['AveragePrice'].mean().plot.bar()
    plt.title(feature+' vs Price')
    plt.xlabel(feature)
    plt.ylabel('Average Price')
    plt.show()

## Numerical Feature
~There are 2 kind of num feat (Discrete and Continues Feature)

In [ ]:
numerical_feature = [feature for feature in data.columns if data[feature].dtype != 'O' and feature not in year_feature]
data[numerical_feature].head()

In [ ]:
discrete_feature = [feature for feature in numerical_feature if len(data[feature].unique()) < 25 and feature not in ['AveragePrice']]
discrete_feature

In [ ]:
continues_feature = [feature for feature in numerical_feature if feature not in discrete_feature]
continues_feature

In [ ]:
for feature in continues_feature:
    data[feature].hist(bins = 25)
    plt.title(feature)
    plt.xlabel(feature)
    plt.show()

In [ ]:
for feature in continues_feature:
    df = data.copy()
    if 0 in df[feature].unique():
        pass
    else:
        df[feature] = np.log(df[feature])
        df[feature].hist(bins = 25)
        plt.title(feature)
        plt.xlabel(feature)
        plt.show()

In [ ]:
for feature in continues_feature:
    if 0 in data[feature].unique():
        pass
    else:
        data[feature] = np.log(data[feature])
        data.boxplot(column = feature)
        plt.title(feature)
        plt.ylabel(feature)
        plt.show()

In [ ]:
data.head()

## Categorical Feature

In [ ]:
categorical_feature = [feature for feature in data.columns if data[feature].dtype == 'O']
data[categorical_feature].head()

In [ ]:
for feature in categorical_feature:
    data.groupby(feature)['AveragePrice'].mean().plot.bar()
    plt.title(feature)
    plt.xlabel(feature)
    plt.ylabel('Average Price')
    plt.show()

## Label Encoder

In [ ]:
from sklearn.preprocessing import LabelEncoder

label = LabelEncoder()
data[categorical_feature] = data[categorical_feature].apply(label.fit_transform)

In [ ]:
data.head()

## Feature Scaling

In [ ]:
feature_scale = [feature for feature in data.columns if feature not in ['AveragePrice']]

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
##data[feature_scale] = scaler.fit_transform(data[feature_scale])
scaler.fit(data[feature_scale])

In [ ]:
scaler.transform(data[feature_scale])

In [ ]:
data = pd.concat([data['AveragePrice'].reset_index(drop = True), pd.DataFrame(scaler.transform(data[feature_scale]), 
                                                                       columns = feature_scale)], axis = 1)

In [ ]:
data.head()

## Feature Selection

In [ ]:
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import Lasso

In [ ]:
X = data.drop('AveragePrice', axis = 1)
y = data['AveragePrice']

In [ ]:
#feat_sel_model = SelectFromModel(Lasso(alpha = 0.005, random_state = 42))
#feat_sel_model.fit(X, y)

In [ ]:
#feat_sel_model.get_support()

In [ ]:
#selected_feat = X.columns[feat_sel_model.get_support()]

In [ ]:
#X = X[selected_feat]

## Build The Model

In [ ]:
#Import Libraries

from sklearn.model_selection import train_test_split, KFold, cross_val_score, RandomizedSearchCV
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import make_scorer, r2_score, mean_squared_error
import math

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
def test_model(model, X_train = X_train, y_train = y_train):
    cv = KFold(n_splits=3, shuffle=True, random_state=42)
    score = cross_val_score(model, X_train, y_train, cv = cv, scoring=make_scorer(r2_score))
    return print('Model Score:', score.mean())

1.Linear Regression

In [ ]:
linear = LinearRegression()
linear.fit(X_train, y_train)
test_model(linear)

2.Lasso

In [ ]:
lasso = Lasso()
lasso.fit(X_train, y_train)
test_model(lasso)

3.Ridge

In [ ]:
ridge = Ridge()
ridge.fit(X_train, y_train)
test_model(ridge)

4.Random Forest

In [ ]:
random = RandomForestRegressor()
random.fit(X_train, y_train)
test_model(random)

5.XGBoost

In [ ]:
xgb = XGBRegressor()
xgb.fit(X_train, y_train)
test_model(xgb)

In [ ]:
## RMSE:

In [ ]:
linear_pred = linear.predict(X_test)
print('Linear Score:', r2_score(y_test, linear_pred))
print('Linear RMSE:', math.sqrt(mean_squared_error(y_test, linear_pred)))

In [ ]:
lasso_pred = lasso.predict(X_test)
print('Lasso Score:', r2_score(y_test, lasso_pred))
print('Lasso RMSE:', math.sqrt(mean_squared_error(y_test, lasso_pred)))

In [ ]:
ridge_pred = ridge.predict(X_test)
print('Ridge Score:', r2_score(y_test, ridge_pred))
print('Ridge RMSE:', math.sqrt(mean_squared_error(y_test, ridge_pred)))

In [ ]:
random_pred = random.predict(X_test)
print('Random Score:', r2_score(y_test, random_pred))
print('Random RMSE:', math.sqrt(mean_squared_error(y_test, random_pred)))

In [ ]:
xgb_pred = xgb.predict(X_test)
print('XGBoost Score:', r2_score(y_test, xgb_pred))
print('XGBoost RMSE:', math.sqrt(mean_squared_error(y_test, xgb_pred)))

In [ ]:
real = pd.DataFrame(np.exp(y_test))
linear = pd.DataFrame(np.exp(linear_pred))
random = pd.DataFrame(np.exp(random_pred))
xgboost = pd.DataFrame(np.exp(xgb_pred))
lasso = pd.DataFrame(np.exp(lasso_pred))
ridge = pd.DataFrame(np.exp(ridge_pred))
df = pd.concat([real, linear, random, xgboost, lasso, ridge], axis = 1)
df.columns = ['Real', 'Linear', 'Random', 'XGBoost', 'Lasso', 'Ridge']
df.dropna(inplace = True)
df.head()

Improve The Model

In [ ]:
n_estimators = [100, 200, 500, 700, 900, 1000, 1100, 1500]
learning_rate = [0.01, 0.05, 0.1, 0.15, 0.2, 0.25]
gamma = [0.0, 0.1, 0.2, 0.3, 0.4]
max_depth = [1,3,5,7,15]
min_child_weight = [1,2,3,4,5]
booster = ['gbtree', 'gblinear']
base_score = [0.25, 0.5, 0.75, 1]

hyper_parameter = {
    'n_estimators': n_estimators,
    'learning_rate': learning_rate,
    'gamma': gamma,
    'max_depth': max_depth,
    'min_child_weight': min_child_weight,
    'booster': booster,
    'base_score': base_score
}

In [ ]:
random_search = RandomizedSearchCV(xgb, param_distributions=hyper_parameter, n_iter=50,
                                  cv= 5, n_jobs=-1, verbose = 3, return_train_score=True,
                                 scoring=make_scorer(r2_score), random_state=42)
random_search.fit(X_train, y_train)

In [ ]:
random_search.best_estimator_

In [ ]:
xgb = XGBRegressor(base_score=1, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0.0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.25, max_delta_step=0, max_depth=15,
             min_child_weight=5, missing=None, monotone_constraints='()',
             n_estimators=500, n_jobs=0, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)
xgb.fit(X_train, y_train)
test_model(xgb)

In [ ]:
xgb_pred = xgb.predict(X_test)
print('XGBoost Score:', r2_score(y_test, xgb_pred))
print('XGBoost RMSE:', math.sqrt(mean_squared_error(y_test, xgb_pred)))

In [ ]:
#The best model is XGboost